# RFP Response Generation

Given template, given context, generate hypothetical RFP response report.

In [ ]:
# download JEDI Cloud RFP Template
!wget "https://imlive.s3.amazonaws.com/Federal%20Government/ID151830346965529215587195222610265670631/HQ0034-18-R-0077.pdf" -O data/jedi_cloud_rfp.pdf

In [ ]:
# microsoft annual report
!wget "https://www.dropbox.com/scl/fi/4v5dx8dc9yqc8k0yw5g4h/msft_10k_2024.pdf?rlkey=jdyfrsoyb18ztlq5msunmibns&st=9w6bdyvn&dl=1" -O data/msft_10k_2024.pdf
# !wget "https://microsoft.gcs-web.com/static-files/1c864583-06f7-40cc-a94d-d11400c83cc8" -O data/msft_10k_2024.pdf

In [ ]:
# azure wikipedia page
!wget "https://www.dropbox.com/scl/fi/7waur8ravmve3fe8nej0k/azure_wiki.pdf?rlkey=icru2w64oylx1p76ftt6y9irv&st=fr87vxob&dl=1" -O data/azure_wiki.pdf

In [ ]:
# azure government slide deck
!wget "https://cdn.ymaws.com/flclerks.site-ym.com/resource/resmgr/2017_Fall_Conf/Presentations/2018-10-12_FCCC_Microsoft_Az.pdf" -O data/azure_gov.pdf

In [ ]:
# microsoft cybersecurity capabilities
!wget "https://www.dropbox.com/scl/fi/qh00xz29rlom4md8ce675/microsoft_ddr.pdf?rlkey=d868nbnsu1ng41y1chw69y64b&st=24iqemb1&dl=1" -O data/msft_ddr.pdf

## Setup

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_parse import LlamaParse

# use our multimodal models for extractions
parser = LlamaParse(
    result_type="markdown",
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="anthropic-sonnet-3.5",
)

In [3]:
from pathlib import Path

data_dir = "data"
files = [
    "azure_gov.pdf",
    "azure_wiki.pdf",
    # "jedi_cloud_rfp.pdf",
    "msft_10k_2024.pdf",
    "msft_ddr.pdf"
]

In [ ]:
file_dicts = {}

for f in files:
    file_base = Path(f).stem
    full_file_path = str(Path(data_dir) / f)
    # md_json_objs = parser.get_json_result(full_file_path)
    # json_dicts = md_json_objs[0]["pages"]
    
    file_docs = parser.load_data(full_file_path)
    
    # attach metadata
    for idx, d in enumerate(file_docs):
        d.metadata["file_path"] = f
        d.metadata["page_num"] = idx + 1

    # image_path = str(Path(out_image_dir) / file_base)
    # image_dicts = parser.get_images(md_json_objs, download_path=image_path)
    file_dicts[f] = {
        "file_path": full_file_path,
        "docs": file_docs
        # "file_path": full_file_path,
        # "json_dicts": json_dicts,
        # "image_path": image_path,
    }

In [4]:
# # TMP
# for f in files:
#     # attach metadata
#     for idx, d in enumerate(file_dicts[f]["docs"]):
#         d.metadata["file_path"] = f
#         d.metadata["page_num"] = idx + 1

In [5]:
# # TMP 
# tmp = parser.load_data("data/azure_gov.pdf")
# print(tmp[0].get_content(metadata_mode="all"))

In [6]:
# print(file_dicts.keys())

# file_dicts["azure_gov.pdf"]

In [19]:
import pickle
pickle.dump(file_dicts, open("tmp_file_dicts.pkl", "wb"))

In [7]:
import pickle
file_dicts = pickle.load(open("tmp_file_dicts.pkl", "rb"))

### Build Indexes

Once the text nodes are ready, we feed into our vector store, which will index these nodes into Chroma (you're welcome to use our other 40+ vector store integrations if you'd like).

In [4]:
!pip install llama-index-vector-stores-chroma

In [30]:
# Run if you want to recreate the index 
!rm -rf storage_rfp_chroma

In [8]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

persist_dir = "storage_rfp_chroma"

vector_store = ChromaVectorStore.from_params(
    collection_name="rfp_docs",
    persist_dir=persist_dir
)
index = VectorStoreIndex.from_vector_store(vector_store)

In [44]:
# !chmod 777 storage_rfp_chroma
!mkdir storage_rfp_chroma

mkdir: storage_rfp_chroma: File exists


**NOTE**: Don't run if you've already inserted the nodes.

In [9]:
# vector_store.clear()
all_nodes = [c for d in file_dicts.values() for c in d["docs"]]

In [10]:
print(all_nodes[0].metadata)

{'file_path': 'azure_gov.pdf', 'page_num': 1}


In [11]:
index.insert_nodes(all_nodes)

Add of existing embedding ID: d7d976fa-be46-4984-966d-afb0478cbe26
Insert of existing embedding ID: d7d976fa-be46-4984-966d-afb0478cbe26


In [90]:
# TMP
tmp_nodes = index.as_retriever(similarity_top_k=5).retrieve("hello")
tmp_nodes

[NodeWithScore(node=TextNode(id_='a2bd510a-2756-4c54-8d29-6186b2e24c93', embedding=None, metadata={'file_path': 'msft_ddr.pdf', 'page_num': 100}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='# Responding with breakthrough innovation continued\n\nExperiences based on large language models, like ChatGPT, have taken the world by storm in the last year. This is in part because they can draw upon a vast store of data, leveraging the massive computing power available today in the cloud. But what I think is more exciting is the real, tangible impact that technology like this can drive. AI, together with the power of cloud and machine learning, has enormous potential. Much like we led the charge on password spray detection, two-factor authentication enforcement, and managed device health, we have an opportunity to demonstrate how responsible AI has the potential to positively transform the security landscape.\n\nAt Microsoft, we are developing first- 

### Define Retrievers

Define retrievers, one for each file. 

In [82]:
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)
from llama_index.core.tools import FunctionTool
from typing import List
from llama_index.core.schema import NodeWithScore

DEFAULT_TOOL_DESCRIPTION = "Retrieves a small set of relevant document chunks from the corpus."

# function tools
def generate_tool(file: str, description: str = DEFAULT_TOOL_DESCRIPTION):
    """Return a function that retrieves only within a given file."""
    filters = MetadataFilters(
        filters=[
            MetadataFilter(
                key="file_path", operator=FilterOperator.EQ, value=file
            ),
        ]
    )
    
    def chunk_retriever_fn(query: str) -> List[NodeWithScore]:
        retriever = index.as_retriever(similarity_top_k=5, filters=filters)
        nodes = retriever.retrieve(query)
        return nodes
    
    # define name as a function of the file
    fn_name = Path(file).stem + "_retrieve"
    
    tool = FunctionTool.from_defaults(fn=chunk_retriever_fn, name=fn_name, description=description)
    
    return tool

# generate tools 
tools = []
for f in files:
    tools.append(generate_tool(f))

In [83]:
tools[0].metadata

ToolMetadata(description='Retrieves a small set of relevant document chunks from the corpus.', name='azure_gov_retrieve', fn_schema=<class 'llama_index.core.tools.utils.azure_gov_retrieve'>, return_direct=False)

In [88]:
# tmp = tools[0]("test").raw_output
# len(tmp)

5

## Build Workflow

The user specifies an RFP document as input. 

Let's build a workflow that can iterate through the extracted keys/questions from the RFP, and fill them out! 

In [16]:
rfp_docs = parser.load_data(Path(data_dir) / "jedi_cloud_rfp.pdf")

Started parsing the file under job_id 7959a846-fa1c-4e8e-a984-b6389320cf6c
.......

In [62]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Context,
    Workflow,
    step,
)
from llama_index.core.llms import LLM
from typing import Optional
from pydantic import BaseModel
from llama_index.core.schema import Document
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.prompts import PromptTemplate
import logging
import json

_logger = logging.getLogger(__name__)
_logger.setLevel(logging.INFO)


# this is the research agent's system prompt, tasked with answering a specific question
AGENT_SYSTEM_PROMPT = """\
You are a research agent tasked with filling out a specific form key/question with the appropriate value, given a bank of context.
You are given a specific form key/question. Think step-by-step and use the existing set of tools to help answer the question.

"""

# This is the prompt tasked with extracting information from an RFP file. 
EXTRACT_KEYS_PROMPT = """\
You are provided an entire RFP document, or a large subsection from it. 

We wish to generate a response to the RFP in a way that adheres to the instructions within the RFP, \
including the specific sections that an RFP response should contain, and the content that would need to go \
into each section.

Your task is to extract out a list of "questions", where each question corresponds to a specific section that is required in the RFP response.
Put another way, after we extract out the questions we will go through each question and answer each one \
with our downstream research assistant, and the combined
question:answer pairs will constitute the full RFP response.

- Make sure the questions are comprehensive and adheres to the RFP requirements.
- Make sure each question is descriptive - this gives our downstream assistant context to fill out the value for that question 
- Extract out all the questions as a list of strings.

"""

class OutputQuestions(BaseModel):
    """List of keys that make up the sections of the RFP response."""
    questions: List[str]


class OutputTemplateEvent(Event):
    docs: List[Document]


class QuestionsExtractedEvent(Event):
    questions: List[str]


class HandleQuestionEvent(Event):
    question: str


class QuestionAnsweredEvent(Event):
    question: str
    answer: str

class CollectedAnswersEvent(Event):
    combined_answers: str
    

class RFPWorkflow(Workflow):
    """RFP workflow."""
    
    def __init__(
        self,
        tools,
        parser: LlamaParse,
        llm: LLM | None = None,
        similarity_top_k: int = 20,
        output_dir: str = "data_out_rfp",
        agent_system_prompt: str = AGENT_SYSTEM_PROMPT,
        **kwargs,
    ) -> None:
        """Init params."""
        super().__init__(**kwargs)
        self.tools = tools
        
        self.parser = parser
        
        self.llm = llm or OpenAI(model="gpt-4o-mini")
        self.similarity_top_k = similarity_top_k
        
        self.output_dir = output_dir
        
        # initialize a Function Calling "research" agent where given a task, it can pull responses from relevant tools 
        self.research_agent = FunctionCallingAgentWorker.from_tools(
            tools, llm=llm, verbose=True, system_prompt=agent_system_prompt
        ).as_agent()
        
        # if not exists, create
        out_path = Path(self.output_dir) / "workflow_output"
        if not out_path.exists():
            out_path.mkdir(parents=True, exist_ok=True)
        
    @step
    async def parse_output_template(self, ctx: Context, ev: StartEvent) -> OutputTemplateEvent:
        # load output template file 
        out_template_path = Path(f"{self.output_dir}/workflow_output/output_template.jsonl")
        if out_template_path.exists():
            with open(out_template_path, "r") as f:
                docs = [Document.parse_obj(json.loads(line)) for line in f]
        else:
            docs = await self.parser.aload_data(ev.rfp_template_path)
            # save output template to file
            with open(out_template_path, "w") as f:
                for doc in docs:
                    f.write(doc.model_dump_json())
                    f.write("\n")

        await ctx.set("output_template", docs)
        return OutputTemplateEvent(docs=docs)
    
    @step
    async def extract_questions(self, ctx: Context, ev: OutputTemplateEvent) -> HandleQuestionEvent:
        docs = ev.docs
        
        # save all_questions to file
        out_keys_path = Path(f"{self.output_dir}/workflow_output/all_keys.txt")
        if out_keys_path.exists():
            with open(out_keys_path, "r") as f:
                output_qs = [q.strip() for q in f.readlines()]
        else:
             # try stuffing all text into the prompt
            all_text = "\n\n".join([d.get_content(metadata_mode="all") for d in docs])
            prompt = PromptTemplate(template=EXTRACT_KEYS_PROMPT)

            try: 
                output_qs = self.llm.structured_predict(
                    OutputQuestions, prompt, context=all_text
                ).questions
            except Exception as e:
                _logger.error(f"Error extracting questions from page: {all_text}")
                _logger.error(e)
                
            with open(out_keys_path, "w") as f:
                f.write("\n".join(output_qs))
        
        await ctx.set("num_to_collect", len(output_qs))

        for question in output_qs:
            ctx.send_event(HandleQuestionEvent(question=question))
        
        return None
    
    @step
    async def handle_question(self, ev: HandleQuestionEvent) -> QuestionAnsweredEvent:
        question = ev.question
        
        # ensure the agent's memory is cleared 
        self.research_agent.reset()
        response = self.research_agent.query(question)
        
        return QuestionAnsweredEvent(question=question, answer=str(response))

    @step
    async def combine_answers(self, ctx: Context, ev: QuestionAnsweredEvent) -> CollectedAnswersEvent:
        num_to_collect = await ctx.get("num_to_collect")
        results = ctx.collect_events(ev, [QuestionAnsweredEvent] * num_to_collect)
        if results is None:
            return None
        
        combined_answers = "\n".join([result.model_dump_json() for result in results])
        # save combined_answers to file
        with open(f"{self.output_dir}/workflow_output/combined_answers.json", "w") as f:
            f.write(combined_answers)

        return CollectedAnswersEvent(combined_answers=combined_answers)

    @step
    async def generate_output(self, ctx: Context, ev: CollectedAnswersEvent) -> StopEvent:
        output_template = await ctx.get("output_template")
        output_template = "\n".join([doc.get_content('none') for doc in output_template])

        prompt = PromptTemplate(
            template=GENERATE_OUTPUT_PROMPT,
        )
        final_output = self.llm.predict(prompt, output_template=output_template, answers=ev.combined_answers)
        # save final_output to file
        with open(f"{self.output_dir}/workflow_output/final_output.md", "w") as f:
            f.write(final_output)

        return StopEvent(result=final_output)

In [63]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")
workflow = RFPWorkflow(
    tools,
    parser=parser,
    llm=llm,
    verbose=True,
    timeout=60.0,
)

In [64]:
# run the agent
response = await workflow.run(rfp_template_path=str(Path(data_dir) / "jedi_cloud_rfp.pdf"))
print(str(response))

Running step parse_output_template
Step parse_output_template produced event OutputTemplateEvent
Running step extract_questions
Step extract_questions produced no event
Running step handle_question
Added user message to memory: What is the overall project scope and objectives as outlined in the RFP?
=== Calling Function ===
Calling function: chunk_retriever_fn with args: {"query": "overall project scope and objectives RFP"}
=== Function Output ===
[]
=== LLM Response ===
It seems that I couldn't find any specific information regarding the overall project scope and objectives outlined in the RFP. If you have access to the RFP document, you might want to check the sections typically titled "Project Scope" or "Objectives" for detailed information. Alternatively, if you provide me with more context or specific details, I can assist you further.
Step handle_question produced event QuestionAnsweredEvent
Running step handle_question
Added user message to memory: What are the specific delivera

CancelledError: 